In [1]:
import torch
import torch.nn as nn
import copy
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
from scipy.io.arff import loadarff
from decimal import Decimal
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

############
# COMPONENTS
############

class Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
        self.rnn1 = nn.LSTM(
            input_size=n_features,
            hidden_size=self.hidden_dim, 
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=self.hidden_dim,
            hidden_size=embedding_dim,
            num_layers=1,
            batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        #print(f'ENCODER input dim: {x.shape}')
        x = x.reshape((batch_size, self.seq_len, self.n_features))
        #print(f'ENCODER reshaped dim: {x.shape}')
        x, (h_n, c_n) = self.rnn1(x)
        #print('h_n = ',h_n.shape)
        #print('c_n = ',c_n.shape)
        #print(f'ENCODER output rnn1 dim: {x.shape}')
        x, (hidden_n, _) = self.rnn2(x)
        #print(f'ENCODER output rnn2 dim: {x.shape}')
        #print(f'ENCODER hidden_n rnn2 dim: {hidden_n.shape}')
        #print(f'ENCODER hidden_n wants to be reshaped to : {(batch_size, self.embedding_dim)}')
        return hidden_n.reshape((batch_size, self.embedding_dim))


class Decoder(nn.Module):
    def __init__(self, seq_len, input_dim=64, n_features=1):
        super(Decoder, self).__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
        self.rnn1 = nn.LSTM(
            input_size=input_dim,
            hidden_size=input_dim,
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=input_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.output_layer = nn.Linear(self.hidden_dim, n_features)

    def forward(self, x):
        batch_size = x.shape[0]
        #print(f'DECODER input dim: {x.shape}')
        x = x.repeat(self.seq_len, self.n_features) # todo testare se funziona con più feature
        #print(f'DECODER repeat dim: {x.shape}')
        x = x.reshape((batch_size, self.seq_len, self.input_dim))
        #print(f'DECODER reshaped dim: {x.shape}')
        x, (hidden_n, cell_n) = self.rnn1(x)
        #print(f'DECODER output rnn1 dim:/ {x.shape}')
        x, (hidden_n, cell_n) = self.rnn2(x)
        #print(f'DECODER output rnn2 dim:/ {x.shape}')
        x = x.reshape((batch_size, self.seq_len, self.hidden_dim))
        return self.output_layer(x)


######
# MAIN
######


class RecurrentAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64, device='cuda', batch_size=10):
        super(RecurrentAutoencoder, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
#load data
dataset = np.load('../data/data_process.npz') 
dataset_normal = dataset['normal'][:,0:140]
dataset_anomaly = dataset['anomaly'][:,0:140]
print(dataset_normal.shape)
print(dataset_anomaly.shape)
dataset_normal = np.expand_dims(dataset_normal,axis=2) #升維
dataset_anomaly = np.expand_dims(dataset_anomaly,axis=2) #升維
print(dataset_normal.shape)
print(dataset_anomaly.shape)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seq_len, n_features = 140, 1
batch_size = 10

################################
validation_split = test_split = 0.15
train_split = 0.85
random_seed = 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset_normal)
indices = list(range(dataset_size)) 
dataset_anomaly_size = len(dataset_anomaly)
anomaly_indices = list(range(dataset_anomaly_size))


train_split = int(train_split * dataset_size)
test_split = int(np.floor(test_split * dataset_size))
validation_split = int(np.floor(validation_split * dataset_size))
print(train_split,test_split,validation_split)

# suffling
#np.random.seed(random_seed)
#np.random.shuffle(indices)
train_indices, val_indices = indices[0:train_split], indices[train_split:]
train_indices, test_indices = train_indices[:train_split-test_split] ,train_indices[train_split-test_split:]


print('train_indices: ', len(train_indices))
print('val_indices: ', len(val_indices))
print('test_indices: ', len(test_indices))


# check all splits have no intersections
assert not [value for value in train_indices if value in test_indices]
assert not [value for value in train_indices if value in val_indices]
assert not [value for value in val_indices if value in test_indices]
##############################


model = RecurrentAutoencoder(seq_len, n_features=n_features, embedding_dim=64, device=device, batch_size=batch_size) 

# Creating PT data samplers and loaders:
train_sampler = SequentialSampler(train_indices) 

valid_sampler = SubsetRandomSampler(val_indices) 
test_sampler = SubsetRandomSampler(test_indices) 
anomaly_sampler = SequentialSampler(anomaly_indices)  

train_loader =DataLoader(dataset_normal,sampler=train_sampler, batch_size=batch_size)
validation_loader = DataLoader(dataset_normal, batch_size=batch_size, sampler=valid_sampler)  
test_loader = DataLoader(dataset_normal, sampler=test_sampler, batch_size=10)  
anomaly_loader = DataLoader(dataset_anomaly,sampler=anomaly_sampler ,batch_size=1)

# start training
n_epochs = 100  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction='mean').to(device) 
history = dict(train=[], val=[])
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 10000.0

for epoch in tqdm(range(1, n_epochs + 1)):
    model = model.train()

    train_losses = []
    val_losses = []
    test_losses = []
    anomaly_losses = []
    
    for i, seq_true in enumerate(train_loader):  
        optimizer.zero_grad()
        seq_true = seq_true.float().cuda()
        seq_pred = model(seq_true)
        loss = criterion(seq_pred, seq_true)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
   
    model = model.eval()
    with torch.no_grad():

        # validation steps
        for i, seq_true in enumerate(validation_loader):
            seq_true = seq_true.to(device)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            val_losses.append(loss.item())

        # normal_test steps
        for i, seq_true in enumerate(test_loader):
            #print(i)
            #print(seq_true[0])
            seq_true = seq_true.to(device)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            test_losses.append(loss.item())

        # anomaly_test steps
        for i, seq_true in enumerate(anomaly_loader):
            seq_true = seq_true.to(device)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            anomaly_losses.append(loss.item())

    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)
    test_loss = np.mean(test_losses)
    #
    anomaly_loss = np.mean(anomaly_losses)
    #自己存的
    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model, '../data/best_model_{}.pt')
    torch.save(model, '../data/last_model_{}.pt')
    
    history['train'].append(train_loss)
    print(f'Epoch {epoch}: train loss {train_loss} {" "*6} val loss {val_loss} {" "*6} test loss {test_loss} {" "*6} anomaly loss {anomaly_loss}')

model.load_state_dict(best_model_wts)

(1400, 140)
(40, 140)
(1400, 140, 1)
(40, 140, 1)
1190 210 210
train_indices:  980
val_indices:  210
test_indices:  210


預測資料

In [ ]:
eval_batch_size = 10 
eval_loss = nn.MSELoss(reduction='none')  

# load trained model
checkpoint_path = '../data/last_model_{}.pt'
model = torch.load(checkpoint_path)
model.eval()

output_data = list()
anomality = list()

In [ ]:
with torch.no_grad():
    for i, data in enumerate(test_loader):  
        #print(i)
        #print(data[0])
        img = data.float().cuda()
        output = model(img)
        sum_loss = eval_loss(output, img).sum([1])
        anomality.append(sum_loss)
        output_data.append(output)
        
    for i, data in enumerate(anomaly_loader):  
        img = data.float().cuda()
        output = model(img)
        sum_loss = eval_loss(output, img).sum([1])
        anomality.append(sum_loss)
        output_data.append(output) 

    
data_pre = torch.cat(anomality, axis=0)
data_pre = data_pre.cpu().detach().numpy()

np.savez('../data/data_pre.npz',normal=data_pre[:210], anomaly=data_pre[210:])    
print(data_pre[:210].shape)
print(data_pre[210:].shape)         
       
output_data = torch.cat(output_data, axis=0)
output_data = output_data.cpu().detach().numpy()
np.savez('../data/output_data.npz',normal=output_data[:210],anomaly=output_data[210:])
print(output_data.shape)